In [ ]:
import pandas as pd
import re
import tkinter as tk
from tkinter import filedialog

# Load messy CSV file
def load_data(filepath: str) -> pd.DataFrame:
    return pd.read_csv(filepath, encoding='utf-8')

# Drop duplicate rows
def drop_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop_duplicates()

# Handle missing values
def handle_missing(df: pd.DataFrame, strategy: str = 'drop') -> pd.DataFrame:
    if strategy == 'drop':
        return df.dropna()
    elif strategy == 'fill_zero':
        return df.fillna(0)
    elif strategy == 'fill_mean':
        return df.fillna(df.mean(numeric_only=True))
    else:
        raise ValueError("Unknown missing value strategy")

# Normalize column names (lowercase, underscores)
def normalize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = [re.sub(r'\W+', '_', col.lower().strip()) for col in df.columns]
    return df

# Convert date columns to datetime
def convert_dates(df: pd.DataFrame, date_cols: list) -> pd.DataFrame:
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

# Remove outliers using IQR method
def remove_outliers(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    for col in cols:
        if pd.api.types.is_numeric_dtype(df[col]):
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower = q1 - 1.5 * iqr
            upper = q3 + 1.5 * iqr
            df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

# Save cleaned data
def save_data(df: pd.DataFrame, filepath: str):
    df.to_csv(filepath, index=False)

# Auto-detect date columns
def detect_date_columns(df: pd.DataFrame) -> list:
    return [col for col in df.columns if 'date' in col.lower() or pd.api.types.is_datetime64_any_dtype(df[col])]

# Auto-detect numeric columns for outlier removal
def detect_outlier_columns(df: pd.DataFrame) -> list:
    return [col for col in df.select_dtypes(include='number').columns if df[col].nunique() > 10]

# Main cleaning pipeline
def clean_csv(input_path: str, output_path: str, date_columns=[], outlier_columns=[]):
    df = load_data(input_path)
    df = normalize_column_names(df)
    df = drop_duplicates(df)
    df = handle_missing(df, strategy='fill_mean')

    if not date_columns:
        date_columns = detect_date_columns(df)
    df = convert_dates(df, date_columns)

    if not outlier_columns:
        outlier_columns = detect_outlier_columns(df)
    df = remove_outliers(df, outlier_columns)

    save_data(df, output_path)
    print(f"Cleaned data saved to {output_path}")

# GUI wrapper for drag-and-drop

def run_gui_cleaner():
    root = tk.Tk()
    root.withdraw()  # Hide main window

    print("Select the CSV file to clean:")
    input_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])

    if not input_path:
        print("No file selected.")
        return

    print("Select where to save the cleaned CSV:")
    output_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

    if not output_path:
        print("No output path selected.")
        return

    clean_csv(
        input_path=input_path,
        output_path=output_path
    )

if __name__ == '__main__':
    run_gui_cleaner()
